# AutoML - Regression

PyCaret
* Main Site - https://pycaret.org/
* Docs - https://pycaret.readthedocs.io/en/latest/

## Table of Contents

* [Setup and Preprocessing](#setup)  
* [Compare Models](#compare)  
* [Create Model](#create)  
* [Tune Model](#tune)  
* [Evaluate Model](#evaluate)  
* [Finalize and Store Model](#finalize_and_store)


## Imports and Global Settings

In [1]:
import os
import sys
import datetime
import json
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from pycaret.regression import RegressionExperiment

here = os.getcwd()
sys.path.append(os.path.join(here, ".."))

from src.utils.modeling_utils import (
    ModelSetup,
    evaluate_reg_model,
    calculate_roi,
    save_model_report,
)

load_dotenv()
RDS_ENDPOINT = os.getenv("RDS_ENDPOINT")
RDS_PASSWORD = os.getenv("RDS_PASSWORD")

# Pandas Settings
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

## Load Data

In [2]:
username = "postgres"
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = "nba_betting"
port = "5432"

# Create the connection string
connection_string = (
    f"postgresql+psycopg2://{username}:{password}@{endpoint}:{port}/{database}"
)

### Games

In [3]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = [
    "game_id",
    "game_datetime",
    "home_team",
    "away_team",
    "open_line",
    "home_score",
    "away_score",
    "game_completed",
    "odds_last_update",
    "scores_last_update",
]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
games_query = f"SELECT {', '.join(features)} FROM games WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    games = pd.read_sql_query(games_query, connection)

### Features

In [4]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = ["game_id", "data"]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
features_query = f"SELECT {', '.join(features)} FROM all_features_json WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    all_features = pd.read_sql_query(features_query, connection)

# Normalize the JSON strings in the 'data' column
expanded_data = pd.json_normalize(all_features["data"])

# Drop the original 'data' column and concatenate the expanded data
all_features = pd.concat([all_features.drop(columns=["data"]), expanded_data], axis=1)

In [5]:
games_features = pd.merge(
    games,
    all_features,
    on="game_id",
    how="left",
    validate="one_to_one",
    suffixes=("", "_drop"),
)
# Drop the columns from df2 (with suffix '_drop')
games_features = games_features[
    games_features.columns.drop(list(games_features.filter(regex="_drop")))
]

<a id='basic_data_overview'></a>

## Basic Data Overview

In [6]:
df = games_features.copy()

In [7]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3894 entries, 0 to 3893
Data columns (total 763 columns):
 #    Column                                        Non-Null Count  Dtype         
---   ------                                        --------------  -----         
 0    game_id                                       3894 non-null   object        
 1    game_datetime                                 3894 non-null   datetime64[ns]
 2    home_team                                     3894 non-null   object        
 3    away_team                                     3894 non-null   object        
 4    open_line                                     3887 non-null   float64       
 5    home_score                                    3882 non-null   float64       
 6    away_score                                    3882 non-null   float64       
 7    game_completed                                3894 non-null   bool          
 8    odds_last_update                              24 non-nul

In [8]:
df.head(10)

game_id       game_datetime home_team away_team  open_line  \
0  20230529BOSMIA 2023-05-29 20:30:00       BOS       MIA       -7.5   
1  20230427ATLBOS 2023-04-27 20:30:00       ATL       BOS        9.5   
2  20230428GSWSAC 2023-04-28 20:00:00       GSW       SAC       -7.0   
3  20230428LALMEM 2023-04-28 22:30:00       LAL       MEM       -5.0   
4  20230504GSWLAL 2023-05-04 21:00:00       GSW       LAL       -7.5   
5  20230502NYKMIA 2023-05-02 19:30:00       NYK       MIA      -10.0   
6  20230502GSWLAL 2023-05-02 22:00:00       GSW       LAL       -5.5   
7  20230429DENPHX 2023-04-29 20:30:00       DEN       PHX       -3.5   
8  20230501BOSPHI 2023-05-01 19:30:00       BOS       PHI      -10.5   
9  20230501DENPHX 2023-05-01 22:00:00       DEN       PHX       -4.0   

   home_score  away_score  game_completed odds_last_update scores_last_update  \
0        84.0       103.0            True              NaT                NaT   
1       120.0       128.0            True              NaT                NaT   
2        99.0       118.0            True              NaT                NaT   
3       125.0        85.0            True              NaT                NaT   
4       127.0       100.0            True              NaT                NaT   
5       111.0       105.0            True              NaT                NaT   
6       112.0       117.0            True              NaT                NaT   
7       125.0       107.0            True              NaT                NaT   
8       115.0       119.0            True              NaT                NaT   
9        97.0        87.0            True              NaT                NaT   

      season  away_ATL  away_BKN  away_BOS  away_CHA  away_CHI  away_CLE  \
0  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
1  2022-2023       0.0       0.0       1.0       0.0       0.0       0.0   
2  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
3  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
4  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
5  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
6  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
7  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
8  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   
9  2022-2023       0.0       0.0       0.0       0.0       0.0       0.0   

   away_DAL  away_DEN  away_DET  away_GSW  away_HOU  away_IND  away_LAC  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
5       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
6       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
7       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
8       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
9       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   away_LAL  away_MEM  away_MIA  away_MIL  away_MIN  away_NOP  away_NYK  \
0       0.0       0.0       1.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       1.0       0.0       0.0       0.0       0.0       0.0   
4       1.0       0.0       0.0       0.0       0.0       0.0       0.0   
5       0.0       0.0       1.0       0.0       0.0       0.0       0.0   
6       1.0       0.0       0.0       0.0       0.0       0.0       0.0   
7       0.0       0.0       0.0       0.0       0.0       0.0 

## Data Preparation

#### Drop Non-Completed Games and Games with No Line

In [9]:
df = df[df["game_completed"] == True]
df = df.dropna(subset=["open_line"])

### Create Targets

In [10]:
df = ModelSetup.add_targets(df)

### Select Features

In [11]:
training_seasons = [x for x in range(2020, 2022)]
training_dates, testing_dates = ModelSetup.choose_dates(training_seasons, [2022], "Reg")
print("Training Dates:")
print(training_dates)
print("Testing Dates:")
print(testing_dates)

Training Dates:
('2020-12-22', '2022-04-10')
Testing Dates:
('2022-10-18', '2023-04-09')


In [12]:
for col in df.columns:
    print(col)

game_id
game_datetime
home_team
away_team
open_line
home_score
away_score
game_completed
odds_last_update
scores_last_update
season
away_ATL
away_BKN
away_BOS
away_CHA
away_CHI
away_CLE
away_DAL
away_DEN
away_DET
away_GSW
away_HOU
away_IND
away_LAC
away_LAL
away_MEM
away_MIA
away_MIL
away_MIN
away_NOP
away_NYK
away_OKC
away_ORL
away_PHI
away_PHX
away_POR
away_SAC
away_SAS
away_TOR
away_UTA
away_WAS
home_ATL
home_BKN
home_BOS
home_CHA
home_CHI
home_CLE
home_DAL
home_DEN
home_DET
home_GSW
home_HOU
home_IND
home_LAC
home_LAL
home_MEM
home_MIA
home_MIL
home_MIN
home_NOP
home_NYK
home_OKC
home_ORL
home_PHI
home_PHX
home_POR
home_SAC
home_SAS
home_TOR
home_UTA
home_WAS
last_5_hv
streak_hv
win_pct_hv
season_type
rest_diff_hv
day_of_season
point_diff_hv
away_team_streak
home_team_streak
away_team_win_pct
home_team_win_pct
point_diff_last_5_hv
pie_away_all_advanced
pie_away_l2w_advanced
pie_home_all_advanced
pie_home_l2w_advanced
l_away_all_traditional
l_away_l2w_traditional
l_home_all_traditio

In [13]:
features_to_use = [
    "open_line",
    "rest_diff_hv",
    "day_of_season",
    "last_5_hv",
    "streak_hv",
    "point_diff_last_5_hv",
    "point_diff_hv",
    "win_pct_hv",
    "plus_minus_home_l2w_opponent",
    "plus_minus_home_l2w_traditional",
    "net_rating_home_l2w_advanced",
    "net_rating_zscore_home_l2w_advanced",
    "e_net_rating_home_l2w_advanced",
    "plus_minus_zscore_home_l2w_traditional",
    "plus_minus_zscore_home_l2w_opponent",
    "e_net_rating_zscore_home_l2w_advanced",
    "plus_minus_percentile_home_l2w_opponent",
    "net_rating_percentile_home_l2w_advanced",
    "plus_minus_percentile_home_l2w_traditional",
    "plus_minus_away_l2w_traditional",
    "e_net_rating_percentile_home_l2w_advanced",
    "plus_minus_away_l2w_opponent",
    "pie_home_l2w_advanced",
    "w_pct_zscore_home_l2w_traditional",
    "e_net_rating_away_l2w_advanced",
    "w_pct_home_l2w_traditional",
    "net_rating_zscore_away_l2w_advanced",
    "e_net_rating_zscore_away_l2w_advanced",
]

In [14]:
df.dropna(subset=features_to_use, inplace=True)

In [15]:
training_df, testing_df, model_report = ModelSetup.create_datasets(
    df, "reg", features_to_use, training_dates, testing_dates, create_report=True
)

In [16]:
print("Training Shape: ", training_df.shape)
print("Testing Shape: ", testing_df.shape)

Training Shape:  (2330, 31)
Testing Shape:  (1199, 31)


### Baselines

In [17]:
training_baseline_via_vegas = model_report["ind_baseline_train"]
testing_baseline_via_vegas = model_report["ind_baseline_test"]

training_baseline_via_mean = model_report["dep_baseline_train"]
testing_baseline_via_mean = model_report["dep_baseline_test"]

print(f"Training Baseline via Vegas: {training_baseline_via_vegas:.2f}")
print(f"Testing Baseline via Vegas: {testing_baseline_via_vegas:.2f}")
print(f"Training Baseline via Mean: {training_baseline_via_mean:.2f}")
print(f"Testing Baseline via Mean: {testing_baseline_via_mean:.2f}")

Training Baseline via Vegas: 10.68
Testing Baseline via Vegas: 9.73
Training Baseline via Mean: 12.17
Testing Baseline via Mean: 10.94


<a id=Regression></a>

## Regression

In [18]:
py_reg = RegressionExperiment()

<a id=setup></a>

### Setup and Preprocessing

The setup process involves a lot of options. Reference the docs below:   
https://pycaret.readthedocs.io/en/latest/api/regression.html#module-pycaret.regression

In [19]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

setup_params_reg = {
    "log_experiment": True,
    "log_profile": False,
    "log_plots": False,
    "experiment_name": f"REG_3_{timestamp}",
    "data": training_df,
    "test_data": testing_df,
    "target": "REG_TARGET",
    "preprocess": False,
    "normalize": False,  # zscore
    "transformation": False,  # yeo-johnson power transform to make data more Gaussian
    "remove_outliers": False,  # using SVD
    "remove_multicollinearity": False,
    "feature_selection": False,
    "pca": False,
    "pca_components": 10,
    "numeric_features": [],
    "ignore_features": ["game_id", "vegas_open_hv"],
}

In [20]:
py_reg.setup(**setup_params_reg)

,Description,Value
0,Session id,3904
1,Target,REG_TARGET
2,Target type,Regression
3,Original data shape,"(3529, 31)"
4,Transformed data shape,"(3529, 29)"
5,Transformed train set shape,"(2330, 29)"
6,Transformed test set shape,"(1199, 29)"
7,Ignore features,2
8,Numeric features,28


2023/10/28 23:53:14 INFO mlflow.tracking.fluent: Experiment with name 'REG_3_20231028235310' does not exist. Creating a new experiment.


<a id=compare></a>

### Compare Models

In [21]:
best_model_reg = py_reg.compare_models(turbo=True, sort="MAE", exclude=["catboost"])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,7.1524,82.2634,9.0623,0.6421,0.8236,1.0065,0.0570
lr,Linear Regression,7.1628,82.5213,9.0771,0.6409,0.8191,1.0085,0.3820
br,Bayesian Ridge,7.1672,82.8086,9.0909,0.6398,0.8260,1.0021,0.0480
huber,Huber Regressor,7.2206,84.1343,9.1613,0.6342,0.8306,0.9975,0.0580
en,Elastic Net,7.4518,90.5306,9.4865,0.6067,0.8481,0.9666,0.0480
lightgbm,Light Gradient Boosting Machine,7.4709,90.7046,9.5145,0.6054,0.8275,1.1026,0.2880
lasso,Lasso Regression,7.4759,90.9061,9.5071,0.6050,0.8477,0.9637,0.0650
gbr,Gradient Boosting Regressor,7.5849,93.2985,9.6516,0.5941,0.8746,1.0126,0.6570
rf,Random Forest Regressor,7.6141,95.7332,9.7781,0.5835,0.8502,1.0488,1.3000
et,Extra Trees Regressor,7.7965,99.1489,9.9495,0.5693,0.8655,1.0845,0.7180


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [22]:
print(best_model_reg)

Ridge(random_state=3904)


<a id=create></a>

### Create Selected Model

In [23]:
model_reg = py_reg.create_model("lr")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,6.6642,74.1224,8.6094,0.6859,0.7989,1.1020
1,6.9792,74.1915,8.6134,0.6205,0.8390,1.1245
2,7.4495,85.1324,9.2267,0.6668,0.8541,1.0504
3,7.0160,77.8096,8.8210,0.6567,0.8223,0.9565
4,7.5495,93.2882,9.6586,0.6129,0.7834,1.0283
5,6.9584,75.6962,8.7004,0.6197,0.8470,0.8218
6,7.1371,81.1914,9.0106,0.6273,0.8459,0.9979
7,7.2487,87.6648,9.3630,0.6095,0.8351,1.1134
8,7.2076,89.1442,9.4416,0.6387,0.7911,1.0088


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

<a id=tune></a>

### Tune Selected Model

In [24]:
tuned_model_reg = py_reg.tune_model(model_reg)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,6.6631,74.1187,8.6092,0.6859,0.7988,1.1019
1,6.9777,74.1597,8.6116,0.6207,0.8405,1.1238
2,7.4498,85.1396,9.2271,0.6668,0.8542,1.0508
3,7.0162,77.8316,8.8222,0.6566,0.8226,0.9563
4,7.5520,93.2693,9.6576,0.6130,0.7857,1.0312
5,6.9576,75.6452,8.6974,0.6200,0.8466,0.8223
6,7.1372,81.1727,9.0096,0.6274,0.8456,0.9979
7,7.2486,87.6702,9.3632,0.6095,0.8356,1.1133
8,7.2004,88.9325,9.4304,0.6395,0.7894,1.0078


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 2 candidates, totalling 20 fits


In [25]:
model_report["details"] = tuned_model_reg.get_params()

<a id=evaluate></a>

### Evaluate Model

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.evaluate_model

In [26]:
py_reg.evaluate_model(tuned_model_reg)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.interpret_model

In [27]:
# py_reg.interpret_model(tuned_model_reg)

In [28]:
train_predictions_reg = py_reg.predict_model(tuned_model_reg, data=training_df)
train_prediction_metrics = py_reg.pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,7.0134,79.0338,8.8901,0.6584,0.8122,0.9886


In [29]:
model_report["train_mae"] = train_prediction_metrics["MAE"][0]
model_report["train_r2"] = train_prediction_metrics["R2"][0]

In [30]:
test_predictions_reg = py_reg.predict_model(tuned_model_reg, data=testing_df)
test_prediction_metrics = py_reg.pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,6.7879,73.2260,8.5572,0.6024,0.8048,1.1696


In [31]:
model_report["test_mae"] = test_prediction_metrics["MAE"][0]
model_report["test_r2"] = test_prediction_metrics["R2"][0]

In [32]:
train_acc_reg, train_closer_to_target_reg, train_prediction_df_reg = evaluate_reg_model(
    train_predictions_reg, "vegas_open_hv", "REG_TARGET", "prediction_label"
)
test_acc_reg, test_closer_to_target_reg, test_prediction_df_reg = evaluate_reg_model(
    test_predictions_reg, "vegas_open_hv", "REG_TARGET", "prediction_label"
)

Prediction is closer to target in 66.95% of cases
Accuracy: 0.7777
Prediction is closer to target in 62.72% of cases
Accuracy: 0.7640


In [33]:
model_report["train_acc_reg"] = train_acc_reg
model_report["test_acc_reg"] = test_acc_reg
model_report["train_ctt"] = train_closer_to_target_reg
model_report["test_ctt"] = test_closer_to_target_reg

In [34]:
roi_results_reg = calculate_roi(test_prediction_df_reg, "actual_side", "pred_side")
roi_results_reg

,Label,Total ROI,Average ROI per Bet
0,"All Bets, Even Amount",63300,52.79
1,"All Bets, Typical Odds",55056,45.92


In [35]:
model_report["roi_all_bets_even_amount_avg"] = roi_results_reg[
    roi_results_reg["Label"] == "All Bets, Even Amount"
]["Average ROI per Bet"].iloc[0]
model_report["roi_all_bets_typical_odds_avg"] = roi_results_reg[
    roi_results_reg["Label"] == "All Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

<a id=finalize_and_store></a>

### Model Finalization and Storage

In [36]:
final_model_reg = py_reg.finalize_model(tuned_model_reg)

In [37]:
platform = "pycaret"
problem_type = "reg"
model_type = "linreg"
datetime_str = model_report["datetime"].strftime("%Y_%m_%d_%H_%M_%S")
model_report["datetime"] = model_report["datetime"].strftime("%Y-%m-%d %H:%M:%S")

model_id = f"{platform}_{problem_type}_{model_type}_{datetime_str}"
model_id

'pycaret_reg_linreg_2023_10_28_23_52_56'

In [38]:
py_reg.save_model(final_model_reg, f"../models/AutoML/{model_id}")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('placeholder', None),
                 ('actual_estimator',
                  LinearRegression(fit_intercept=False, n_jobs=-1))]),
 '../models/AutoML/pycaret_reg_linreg_2023_10_28_23_52_56.pkl')

In [39]:
model_report["platform"] = platform
model_report["model_type"] = model_type
model_report["model_id"] = model_id

In [40]:
model_report

{'datetime': '2023-10-28 23:52:56',
 'problem_type': 'reg',
 'target': 'REG_TARGET',
 'features': ['open_line',
  'rest_diff_hv',
  'day_of_season',
  'last_5_hv',
  'streak_hv',
  'point_diff_last_5_hv',
  'point_diff_hv',
  'win_pct_hv',
  'plus_minus_home_l2w_opponent',
  'plus_minus_home_l2w_traditional',
  'net_rating_home_l2w_advanced',
  'net_rating_zscore_home_l2w_advanced',
  'e_net_rating_home_l2w_advanced',
  'plus_minus_zscore_home_l2w_traditional',
  'plus_minus_zscore_home_l2w_opponent',
  'e_net_rating_zscore_home_l2w_advanced',
  'plus_minus_percentile_home_l2w_opponent',
  'net_rating_percentile_home_l2w_advanced',
  'plus_minus_percentile_home_l2w_traditional',
  'plus_minus_away_l2w_traditional',
  'e_net_rating_percentile_home_l2w_advanced',
  'plus_minus_away_l2w_opponent',
  'pie_home_l2w_advanced',
  'w_pct_zscore_home_l2w_traditional',
  'e_net_rating_away_l2w_advanced',
  'w_pct_home_l2w_traditional',
  'net_rating_zscore_away_l2w_advanced',
  'e_net_rating_zsc

In [41]:
save_model_report(model_report)